# Debugging a planning process with SIADEX

SIADEX is an Hierarchical Task Network (HTN) planner with its own planning language (HPDL), but its integration with UPF supports also domains and problem written in HDDL and with the UPF framework.

The [SIADEX planner](https://github.com/IgnacioVellido/HPDL-Planner) has an especial operation mode that works like a debugger, where users can control the planning process, evaluate conditions, watch/alter the planning live state and attach breakpoints. In this notebook we will show the interactive debugging mode that is provided by the SIADEX UPF integration.

In case you are not familiar with what SIADEX or hierarchical planning is, take a look at the [SIADEX](siadex.ipynb) notebook or the [Wikipedia page](https://en.wikipedia.org/wiki/Hierarchical_task_network).

First, we will need to install SIADEX and register it as an UPF engine

####  _______ Begin Installation____________

**Disclaimer** : The installation steps are only needed until up_siadex is published on pypi and the changes on the unified-planning fork are merged into the origin package

In [ ]:
# Make sure the following packages are installed on the system: python-dev libreadline-dev. Those are needed for the execution of Siadex
!apt-get update
!apt-get install -y python-dev libreadline-dev

Leyendo lista de paquetes... Hecho
E: No se pudo abrir el fichero de bloqueo «/var/lib/apt/lists/lock» - open (13: Permiso denegado)
E: No se pudo bloquear el directorio /var/lib/apt/lists/
W: Se produjo un problema al desligar el fichero /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permiso denegado)
W: Se produjo un problema al desligar el fichero /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permiso denegado)
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: No se pudo obtener el bloqueo de la interfaz dpkg (/var/lib/dpkg/lock-frontend). ¿Es usted superusuario?


In [ ]:
# Clonning the repos
!git clone https://github.com/UGR-IntelligentSystemsGroup/unified-planning.git
!git clone https://github.com/UGR-IntelligentSystemsGroup/up-siadex.git

fatal: la ruta de destino 'unified-planning' ya existe y no es un directorio vacío.
fatal: la ruta de destino 'up-siadex' ya existe y no es un directorio vacío.


In [ ]:
# Install the packages
# %%capture
%pip install -e ./unified-planning
%pip install -e ./up-siadex

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///mnt/e/newDesktop/void/projects/siadex/up-siadex/Notebooks/unified-planning
  Preparing metadata (setup.py) ... done
  Attempting uninstall: unified-planning
    Found existing installation: unified-planning 0.4.2
    Uninstalling unified-planning-0.4.2:
      Successfully uninstalled unified-planning-0.4.2
  Running setup.py develop for unified-planning
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///mnt/e/newDesktop/void/projects/siadex/up-siadex/Notebooks/up-siadex
  Preparing metadata (setup.py) ... done
  Attempting uninstall: up-siadex
    Found existing installation: up-siadex 0.0.2
    Uninstalling up-siadex-0.0.2:
      Successfully uninstalled up-siadex-0.0.2
  Running setup.py develop for up-siadex
Note: you may need to restart the kernel to use updated packages.


####  _______ End Installation____________

In [39]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus

# Register SIADEX
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

## Creating a planning problem

First we need to create a UPF Problem instance that we want to solve. In this case, we will read the HDDL __HTN-transport__ domain, which describes how a fleet of trucks should move packages across multiple cities

In [40]:
reader = PDDLReader()
problem = reader.parse_problem("./unified-planning/unified_planning/test/pddl/htn-transport/domain.hddl",
                               "./unified-planning/unified_planning/test/pddl/htn-transport/problem.hddl")

For this specific instance, we have vehicles and locations. The goal is to move package-0 to city-loc-0 and package-1 to city-loc-2 using the vehicles.

We can see the goal via the __task_network__ attribute of our problem instance.

In [41]:
problem.task_network

task network {
  subtasks = [
    _t51: deliver(package-0, city-loc-0)
        time_constraints = [
        ]
    _t52: deliver(package-1, city-loc-2)
        time_constraints = [
        ]
  ]
}

## Debugging

Now we are ready to plan. To start our debugging mode, we create an instance of the SIADEX planner using the OneShotPlanner interface and call the __debugger__ method.
This will return an class instance with methods that will work as an interface between the user and the SIADEX internal debugger.

In [42]:
siadex = OneshotPlanner(name='siadex')
debugger = siadex.debugger()
debugger.debug(problem)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_395/182887877.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

Debugger stopped
debug:> debug:> break (siadex_debugger_stop )
__________________________________________________

(*** 1 ***) Initializating planning process...
(*** 1 ***) Initializating python...
(*** 1 ***) Depth: 1
(*** 1 ***) Selecting task to expand from agenda.


To start the debugging process, we call the __debug__ method with the UPF problem that we want to solve. Through the rest of this notebook we will show the available commands and the output of each one. 

There are three ways of advancing through the planner process. We can use the __next()__ or __nexp()__ methods to move one step or move until the next task is found, respectively; or we can use __continue_run()__ to forward until the next breakpoint or the planning process ends.

### Planner internal state

To get the actual planning state of the debugger we can use the method __state()__. This returns the list of fluents that are currently active and true.

In [43]:
state = debugger.state()
state

[road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

### Planner agenda

The agenda represents the actual process of the debugger expanding nodes. It returns a dict of nodes, where each node has the following information:

 - __subtask__: The task/action that is susceptible to be expanded with the instanced object or named variables.
 - __status__: Agenda, pending, closed, action.
 - __expanded__: Boolean flag indicating if the node has been expanded or not, that is, if the planner has already opened the node to get its children.
 - __successors__: A list of the children nodes.

An special breakpoint and action __(siadex_debugger_stop)__ have been automatically added. This is needed for the correct operation of the debugger. The breakpoint and action does not interfere with the original problem and solution and can be ignored. 

In [44]:
nodes = debugger.agenda()
nodes

{0: 0: (root ) 
                 status: root, expanded: root, 
                 successors: [1, 2, 3],
 1: 1: (deliver [package-0, city-loc-0]) 
                 status: agenda, expanded: unexpanded, 
                 successors: [],
 2: 2: (deliver [package-1, city-loc-2]) 
                 status: agenda, expanded: unexpanded, 
                 successors: [],
 3: 3: (siadex_debugger_stop []) 
                 status: agenda, expanded: unexpanded, 
                 successors: []}

In [45]:
nodes[1].subtask

_t54: deliver(package-0, city-loc-0)
        time_constraints = [
        ]

With agenda_tree() we can take a look to the agenda in a more comfortable way.

In [46]:
debugger.agenda_tree()

0: (root ) status: root, expanded: root
    1: (deliver [package-0, city-loc-0]) status: agenda, expanded: unexpanded
    2: (deliver [package-1, city-loc-2]) status: agenda, expanded: unexpanded
    3: (siadex_debugger_stop []) status: agenda, expanded: unexpanded


Advance one step in the debug process. 

In [47]:
debugger.nexp()

debug:> nexp
__________________________________________________
(*** 1 ***) Expanding: [0] (deliver package_0 city_loc_0)

(*** 1 ***) Selecting a candidate task.
(*** 1 ***) Found: 1 candidates (left).
      [0] :task (deliver ?p ?l)


### Adding breakpoints

Breakpoints lets us set points where we want the debugger to stop, and then use the continue_run(), next() or nexp() methods to go quickly towards them.
We can add breakpoints to a fluent, expressions, actions and task. 

First we get a fluent and objects from the original problem to work with them.

In [48]:
drive = problem.action("drive")
road = problem.fluent("road")
city_0 = problem.object("city-loc-0")
city_1 = problem.object("city-loc-1")
road, city_0, city_1, drive

(bool road[l1=location, l2=location],
 city-loc-0,
 city-loc-1,
 action drive(vehicle - locatable v, location l1, location l2) {
     preconditions = [
       (at(v, l1) and road(l1, l2))
     ]
     effects = [
       at(v, l1) := false
       at(v, l2) := true
     ]
     simulated effect = None
   })

With add_break we can define breakpoints where the debugger will stop when exploring

In [50]:
debugger.add_break(road)

debug:> break (road ?l1 ?l2)
__________________________________________________


{0: {'id': 0, 'enabled': True, 'node': '(siadex_debugger_stop)'},
 1: {'id': 1, 'enabled': True, 'node': '(road ?l1 ?l2)'}}

In [51]:
debugger.add_break(drive)

debug:> break (drive ?v ?l1 ?l2)
__________________________________________________


{0: {'id': 0, 'enabled': True, 'node': '(siadex_debugger_stop)'},
 1: {'id': 1, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 2: {'id': 2, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

And with enable_break and disable_break, we can enable a disable breakpoints

In [52]:
debugger.disable_break(1)

{0: {'id': 0, 'enabled': True, 'node': '(siadex_debugger_stop)'},
 1: {'id': 1, 'enabled': False, 'node': '(road ?l1 ?l2)'},
 2: {'id': 2, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

In [53]:
debugger.enable_break(1)

{0: {'id': 0, 'enabled': True, 'node': '(siadex_debugger_stop)'},
 1: {'id': 1, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 2: {'id': 2, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

In [54]:
debugger.list_break()

{0: {'id': 0, 'enabled': True, 'node': '(siadex_debugger_stop)'},
 1: {'id': 1, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 2: {'id': 2, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

### Modifying the planner state

The planner state is the set of predicates and fluents that are currently true.

With an expression we can alter the live state of the debugger 

In [55]:
f_road = road(city_0, city_1)
f_road.Not()

(not road(city-loc-0, city-loc-1))

In [56]:
# Actual state
debugger.state()
# We can see road(city-loc-0, city-loc-1) on the state

[road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

With a fluent and the method __apply_effect()__ we can add new clauses or deny from the state 

In [57]:
debugger.apply_effect(f_road.Not())
# Deny (road(city-loc-0, city-loc-1))

[road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

In [58]:
debugger.apply_effect(f_road)
# Add (road(city-loc-0, city-loc-1))

[road(city-loc-1, city-loc-0),
 road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

### Evaluate preconditions

With the method eval_preconditions() we can evaluate on the actual state the preconditions of an action or fluents. It returns a list of valid parameters for that expression. 

In [59]:
debugger.eval_preconditions(road)

[{'l1': city-loc-2, 'l2': city-loc-1},
 {'l1': city-loc-1, 'l2': city-loc-2},
 {'l1': city-loc-0, 'l2': city-loc-1},
 {'l1': city-loc-1, 'l2': city-loc-0}]

In [60]:
debugger.eval_preconditions(drive)

[{vehicle - locatable v: truck-0,
  location l1: city-loc-2,
  location l2: city-loc-1}]

Avance 10 tasks in the planning process

In [61]:
debugger.nexp(10)

debug:> nexp
__________________________________________________
(*** 1 ***) Selecting a method to expand from compound task.
      :task (deliver ?p ?l)
(*** 1 ***) Found: 1 methods to expand (left).
      [0] 
      (:method m_deliver
      :precondition
      ( )
      :tasks (
         (get_to ?v ?l1)
         (load ?v ?l1 ?p)
         (get_to ?v ?l2)
         (unload ?v ?l2 ?p)
      )
      )
Breakpoint 1 reached.
debug:> nexp
__________________________________________________
(*** 1 ***) Expanding method: m_deliver
(*** 1 ***) Working in task:
(:task deliver
   :parameters ( ?p - package ?l - location)
   (:method m_deliver
   :precondition
   ( )
   :tasks (
      (get_to ?v ?l1)
      (load ?v ?l1 package_0)
      (get_to ?v ?l2)
      (unload ?v ?l2 package_0)
   )
   )
)

(*** 1 ***) Using method: m_deliver
(*** 1 ***) No preconditions.
Selecting unification: 
(*** 2 ***) Depth: 2
(*** 2 ***) Selecting task to expand from agenda.
Breakpoint 1 reached.
debug:> nexp
___________

With plan() we can obtain the result plan that has been already discovered  

In [62]:
debugger.plan()

[drive(truck-0, city-loc-2, city-loc-1)]

In [63]:
debugger.agenda_tree()

0: (root ) status: root, expanded: root
    2: (deliver [package-1, city-loc-2]) status: agenda, expanded: unexpanded
    3: (siadex_debugger_stop []) status: agenda, expanded: unexpanded
    9: (drive [truck-0, city-loc-2, city-loc-1]) status: pending, expanded: action
        5: (load [truck-0, l, package-0]) status: pending, expanded: unexpanded
            6: (get-to [truck-0, l]) status: pending, expanded: unexpanded
                7: (unload [truck-0, l, package-0]) status: pending, expanded: unexpanded


With __force_run()__ we can directly interact with the debugger using string commands

In [65]:
debugger.force_run("help break")

debug:> help break
__________________________________________________
Command: `break'. Shortcut `b'

Description: Manages the stablished breakpoints.
	`break':	Lists all defined breakpoints.
	`break <number>':	Prints breakpoint whith given id.
	`break <predicate>':	Defines a new breakpoint. <predcate> can be a task definition or a simple predicate.
See also: `watch', `disable', `enable'.


In [66]:
debugger.list_break()

{0: {'id': 0, 'enabled': True, 'node': '(siadex_debugger_stop)'},
 1: {'id': 1, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 2: {'id': 2, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

Let's disable all the breakpoints

In [67]:
for i,_ in debugger.list_break().items():
    debugger.disable_break(i)

With __continue_run()__ we can advance until the next breakpoint or goal is reached

In [68]:
debugger.continue_run()

debug:> continue
__________________________________________________
(*** 4 ***) Expanding method: m_load
(*** 4 ***) Working in task:
(:task load
   :parameters ( ?v - vehicle ?l - location ?p - package)
   (:method m_load
   :precondition
   ( )
   :tasks (
      (pick_up truck_0 ?l1 package_0 ?s1 ?s2)
   )
   )
)

(*** 4 ***) Using method: m_load
(*** 4 ***) No preconditions.
Selecting unification: 
(*** 5 ***) Depth: 5
(*** 5 ***) Selecting task to expand from agenda.
(*** 5 ***) Expanding: [9] (pick_up truck_0 ?l1 package_0 ?s1 ?s2)

(*** 5 ***) Selecting a candidate task.
(*** 5 ***) Found: 1 candidates (left).
      [0] :action (pick_up truck_0 ?l1 package_0 ?s1 ?s2)
(*** 5 ***) Solving action:
(:action pick_up
 :parameters ( truck_0 ?l1 - location package_0 ?s1 - capacity_number ?s2 - capacity_number)
 :precondition
   (and
      (and
         (at_ truck_0 ?l1)
         (at_ package_0 ?l1)
         (capacity_predecessor ?s1 ?s2)
         (capacity truck_0 ?s2)
      )

   )

 :e

# Stopping the debugger
The debugger has ended when it stops on the breakpoint 0.

With __plan()__ we can obtain the final plan.

In [69]:
debugger.plan()

[drive(truck-0, city-loc-2, city-loc-1), pick-up(truck-0, city-loc-1, package-0, capacity-0, capacity-1), drive(truck-0, city-loc-1, city-loc-0), drop(truck-0, city-loc-0, package-0, capacity-0, capacity-1), drive(truck-0, city-loc-0, city-loc-1), pick-up(truck-0, city-loc-1, package-1, capacity-0, capacity-1), drive(truck-0, city-loc-1, city-loc-0), drop(truck-0, city-loc-0, package-1, capacity-0, capacity-1)]

With __stop()__ we can stop the debugger.

In [43]:
debugger.stop()

Debugger stopped
